### Importing The Libraries

In [1]:
import yaml
import spacy
import plac
import random
from pathlib import Path
from spacy.util import minibatch , compounding
from spacy.training import Example

### Train Data

In [2]:
data_file = open('..\data\entities.yaml').read()
entities = yaml.safe_load(data_file)


train_data = []


for entity in entities['train_data']:
    train_data.append(entity)

### Create the Model

In [3]:
# Define our Variables
ner_model = None
output_dir=Path("../Name Entity Recognition")
n_iter=100

# Load the Model
if ner_model is not None:
    nlp = spacy.load(ner_model)  
    print("Loaded model '%s'" % ner_model)
else:
    nlp = spacy.blank('en')  
    print("Created blank 'en' model")

# Set up the pipeline
if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe('ner', last=True)
else:
    ner = nlp.get_pipe('ner')

Created blank 'en' model


### Train the Recognizer

In [4]:
for _, annotations in train_data:
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])

disable_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']

with nlp.disable_pipes(*disable_pipes):
    optimizer = nlp.begin_training()
    
    for i in range(n_iter):
        random.shuffle(train_data)
        losses = {}
        
        batches = minibatch(train_data , size = compounding(1.0,16.0,1.001))
        for batch in batches:
            texts, annotations = zip(*batch)
            
            example = []
            # Update the model with iterating each text
            for i in range(len(texts)):
                doc = nlp.make_doc(texts[i])
                example.append(Example.from_dict(doc, annotations[i]))
            
                # Update the model
                nlp.update(
                           example,
                           drop=0.6,
                           sgd = optimizer,
                           losses=losses
                           )
            print(losses)

{'ner': 8.099999248981476}
{'ner': 13.576015830039978}
{'ner': 20.196689903736115}
{'ner': 27.710106551647186}
{'ner': 34.21516090631485}
{'ner': 39.733898997306824}
{'ner': 44.344391882419586}
{'ner': 50.62965738773346}
{'ner': 56.118127942085266}
{'ner': 63.09629625082016}
{'ner': 68.80435621738434}
{'ner': 73.94676148891449}
{'ner': 78.66372764110565}
{'ner': 82.99029210209846}
{'ner': 87.37623426318169}
{'ner': 91.29621097445488}
{'ner': 94.28457260131836}
{'ner': 98.30762651562691}
{'ner': 101.08827084302902}
{'ner': 105.91298598051071}
{'ner': 109.99953922629356}
{'ner': 113.21376098692417}
{'ner': 116.64675168693066}
{'ner': 119.05729247257113}
{'ner': 123.83085898682475}
{'ner': 127.80907666124403}
{'ner': 129.72084509395063}
{'ner': 132.85859899781644}
{'ner': 136.77333076670766}
{'ner': 138.89513360522687}
{'ner': 140.95030049979687}
{'ner': 145.99842014885508}
{'ner': 149.1154109367635}
{'ner': 150.91431179619394}
{'ner': 152.657887555426}
{'ner': 154.56533904070966}
{'ner':

c:\Users\Mohamed Nabil\AppData\Local\Programs\Python\Python39\lib\site-packages\spacy\training\iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "is it going to rain in cairo" with entities "[[15, 19, 'weather_descriptor'], [23, 38, 'locatio...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(


{'ner': 404.5219994418876}
{'ner': 406.158998654093}
{'ner': 408.2221661888616}
{'ner': 409.6328474673064}
{'ner': 414.59146873081045}
{'ner': 418.8828827247926}
{'ner': 420.740816678197}
{'ner': 422.47516268996696}
{'ner': 425.1121303985565}
{'ner': 426.5323067198551}
{'ner': 428.25766247911815}
{'ner': 433.6273209471765}
{'ner': 435.5671981983396}
{'ner': 438.30350086284307}
{'ner': 439.8452255558732}
{'ner': 441.5060042916375}
{'ner': 446.5763960758268}
{'ner': 448.0287768429189}
{'ner': 449.9631259919188}
{'ner': 454.4505772785851}
{'ner': 456.7328528016151}
{'ner': 460.29886382741824}
{'ner': 463.17272482410044}
{'ner': 465.2783159359169}
{'ner': 468.78449861768604}
{'ner': 470.486127443648}
{'ner': 472.4040739292759}
{'ner': 473.9272678865018}
{'ner': 477.2507833505879}
{'ner': 479.1933880299621}
{'ner': 483.5615310606372}
{'ner': 485.03881345133414}
{'ner': 486.97439968123217}
{'ner': 489.35525540524395}
{'ner': 491.48342777647485}
{'ner': 492.72134076616203}
{'ner': 497.5783457

### Save the Model

In [5]:
if output_dir is not None:
    output_dir = Path(output_dir)
    if not output_dir.exists():
        output_dir.mkdir()
    nlp.to_disk(output_dir)
    print("Saved model to", output_dir)   

Saved model to ..\Name Entity Recognition
